In [1]:
import re
import pandas as pd
import numpy as np

# PREPROCESSING

In [2]:
df = pd.read_csv('merged_file.csv')

## check duplicates

In [3]:
df.duplicated().sum()

np.int64(4394)

In [4]:
df = df.drop_duplicates(keep='first').reset_index(drop=True)
df.shape

(44714, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44714 entries, 0 to 44713
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               44714 non-null  object
 1   brand            44714 non-null  object
 2   price            44714 non-null  object
 3   start_time       44714 non-null  object
 4   start_day        44714 non-null  object
 5   end_time         44714 non-null  object
 6   end_day          44714 non-null  object
 7   trip_time        44714 non-null  object
 8   destination      44714 non-null  object
 9   checked_baggage  36071 non-null  object
 10  hand_luggage     36071 non-null  object
 11  crawl_date       44714 non-null  object
dtypes: object(12)
memory usage: 4.1+ MB


## Take_place, Destination

In [6]:
df['destination'].unique()

array(['Nha Trang (CXR)\nSân bay Cam Ranh',
       'Đà Nẵng (DAD)\nSân bay Đà Nẵng',
       'Hà Nội (HAN)\r\nSân bay Nội Bài',
       'Hà Nội (HAN)\r\nSân bay Nội Bài\r\nNhà ga 1',
       'Hải Phòng (HPH)\nSân bay quốc tế Cát Bi',
       'Phú Quốc (PQC)\nSân bay Phú Quốc'], dtype=object)

In [7]:
df['destination'] = df['destination'].replace(
    'Hà Nội (HAN)\r\nSân bay Nội Bài\r\nNhà ga 1',
    'Hà Nội (HAN)\r\nSân bay Nội Bài'
)
df['destination'].value_counts()

destination
Hà Nội (HAN)\r\nSân bay Nội Bài            19282
Đà Nẵng (DAD)\nSân bay Đà Nẵng             12801
Phú Quốc (PQC)\nSân bay Phú Quốc            5273
Hải Phòng (HPH)\nSân bay quốc tế Cát Bi     4186
Nha Trang (CXR)\nSân bay Cam Ranh           3172
Name: count, dtype: int64

## brand

In [8]:
df['brand'].nunique(), df['brand'].unique()

(4,
 array(['VietJet Air', 'Vietnam Airlines', 'Bamboo Airways',
        'Vietravel Airlines'], dtype=object))

## price

In [9]:
df['price'].head()

0    1.892.258 VND/khách
1    1.431.413 VND/khách
2    1.327.929 VND/khách
3    1.283.783 VND/khách
4    1.285.778 VND/khách
Name: price, dtype: object

In [10]:
# Clean and convert the price column
df['price'] = df['price'].str.extract(r'([\d\.]+)')
df['price'] = df['price'].str.replace('.', '', regex=False).astype(int)
df['price'].head()

0    1892258
1    1431413
2    1327929
3    1283783
4    1285778
Name: price, dtype: int64

## time

In [11]:
df['start_time'] = df['start_time'].str.replace('h', ':')
df['end_time'] = df['end_time'].str.replace('h', ':')

In [12]:
df['start_hour'] = df['start_time'].str.split(':').str[0].astype(int)
df['end_hour'] = df['end_time'].str.split(':').str[0].astype(int)
df[['start_time', 'start_hour', 'end_time', 'end_hour']].head()

,start_time,start_hour,end_time,end_hour
0,10:20,10,11:20,11
1,10:20,10,11:20,11
2,10:20,10,11:20,11
3,10:20,10,11:20,11
4,10:20,10,11:20,11


In [13]:
df['start_hour'] = pd.cut(
    df['start_hour'],
    bins=[0, 3, 9, 15, 21, 24],
    labels=['EarlyMorning', 'Morning', 'Afternoon', 'Evening', 'LateNight'],
    include_lowest=True
)
df['end_hour'] = pd.cut(
    df['end_hour'],
    bins=[0, 3, 9, 15, 21, 24],
    labels=['EarlyMorning', 'Morning', 'Afternoon', 'Evening', 'LateNight'],
    include_lowest=True
)
df[['start_time', 'start_hour', 'end_time', 'end_hour']].head()

,start_time,start_hour,end_time,end_hour
0,10:20,Afternoon,11:20,Afternoon
1,10:20,Afternoon,11:20,Afternoon
2,10:20,Afternoon,11:20,Afternoon
3,10:20,Afternoon,11:20,Afternoon
4,10:20,Afternoon,11:20,Afternoon


In [14]:
time_parts = df['trip_time'].str.extract(
    r'(?:(?P<hour>\d+)h)?\s*(?:(?P<minute>\d+)m)?')
time_parts = time_parts.astype(float).fillna(0)

df['trip_mins'] = time_parts['hour'] * 60 + time_parts['minute']

df[['trip_time', 'trip_mins']].value_counts()

trip_time  trip_mins
2h 10m     130.0        11287
2h 5m      125.0         8404
1h 25m     85.0          6576
1h 20m     80.0          4871
1h 0m      60.0          4751
2h 0m      120.0         3341
1h 5m      65.0          2602
1h 15m     75.0           905
1h 30m     90.0           760
55m        55.0           447
1h 55m     115.0          409
45m        45.0           334
2h 15m     135.0           26
2h 25m     145.0            1
Name: count, dtype: int64

In [15]:
df['trip_mins'] = df['trip_mins'].astype(int)
df.drop(columns=['start_time', 'end_time', 'trip_time'], inplace=True)

## day

In [16]:
df['start_day'].unique(), df['end_day'].unique()

(array(['01 thg 5', '08 thg 5', '02 thg 5', '03 thg 5', '04 thg 5',
        '05 thg 5', '06 thg 5', '07 thg 5', '09 thg 5', '10 thg 5',
        '11 thg 5', '22 thg 4', '23 thg 4', '24 thg 4', '25 thg 4',
        '26 thg 4', '27 thg 4', '28 thg 4', '29 thg 4', '30 thg 4',
        '21 thg 4'], dtype=object),
 array(['01 thg 5', '08 thg 5', '02 thg 5', '03 thg 5', '04 thg 5',
        '05 thg 5', '06 thg 5', '07 thg 5', '09 thg 5', '10 thg 5',
        '11 thg 5', '22 thg 4', '23 thg 4', '24 thg 4', '25 thg 4',
        '26 thg 4', '27 thg 4', '28 thg 4', '29 thg 4', '30 thg 4',
        '21 thg 4', '12 thg 5'], dtype=object))

In [17]:
df['crawl_date'].unique()

array(['14-04-2025', '15-04-2025', '16-04-2025', '17-04-2025',
       '18-04-2025', '11-04-2025', '12-04-2025', '13-04-2025',
       '19-04-2025', '24-04-2025', '29-04-2025', '01-05-2025',
       '20-04-2025', '21-04-2025', '22-04-2025', '23-04-2025',
       '25-04-2025', '26-04-2025', '27-04-2025', '28-04-2025',
       '30-04-2025', '02-05-2025', '03-05-2025', '04-05-2025',
       '05-05-2025', '06-05-2025', '07-05-2025', '08-05-2025',
       '09-05-2025', '07-04-2025', '08-04-2025', '09-04-2025',
       '10-04-2025', '10-05-2025'], dtype=object)

In [18]:
def convert_vn_date(date_str, year=2025):
    day, month = date_str.strip().split(' thg ')
    dt = pd.to_datetime(f"{day}-{int(month):02d}-{year}", dayfirst=True)
    return dt


df['start_day'] = df['start_day'].apply(lambda x: convert_vn_date(x, 2025))
df['end_day'] = df['end_day'].apply(lambda x: convert_vn_date(x, 2025))
df[['start_day', 'end_day']].head(), df['start_day'].dtype, df['end_day'].dtype

(   start_day    end_day
 0 2025-05-01 2025-05-01
 1 2025-05-01 2025-05-01
 2 2025-05-01 2025-05-01
 3 2025-05-01 2025-05-01
 4 2025-05-01 2025-05-01,
 dtype('<M8[ns]'),
 dtype('<M8[ns]'))

In [19]:
holidays = [
    pd.Timestamp('2025-04-30').date(),
    pd.Timestamp('2025-05-01').date(),
]

nearby_holidays = [
    pd.Timestamp('2025-04-29').date(),
    pd.Timestamp('2025-05-02').date(),
    pd.Timestamp('2025-05-03').date(),
    pd.Timestamp('2025-05-04').date(),
]


def is_holiday(date):
    d = date.date()
    if d in holidays:
        return 3
    elif d in nearby_holidays:
        return 2
    elif d.weekday() >= 4:  # Friday = 4
        return 1
    else:
        return 0


df['is_holiday'] = df['start_day'].apply(is_holiday)
df[['start_day', 'is_holiday']].value_counts().head(5)

start_day   is_holiday
2025-05-05  0             2464
2025-04-27  1             2424
2025-05-02  2             2412
2025-04-28  0             2366
2025-05-03  2             2355
Name: count, dtype: int64

In [20]:
df['start_day'].dtype

dtype('<M8[ns]')

In [21]:
df['crawl_date'].unique()

array(['14-04-2025', '15-04-2025', '16-04-2025', '17-04-2025',
       '18-04-2025', '11-04-2025', '12-04-2025', '13-04-2025',
       '19-04-2025', '24-04-2025', '29-04-2025', '01-05-2025',
       '20-04-2025', '21-04-2025', '22-04-2025', '23-04-2025',
       '25-04-2025', '26-04-2025', '27-04-2025', '28-04-2025',
       '30-04-2025', '02-05-2025', '03-05-2025', '04-05-2025',
       '05-05-2025', '06-05-2025', '07-05-2025', '08-05-2025',
       '09-05-2025', '07-04-2025', '08-04-2025', '09-04-2025',
       '10-04-2025', '10-05-2025'], dtype=object)

In [22]:
df['crawl_date'] = pd.to_datetime(
    df['crawl_date'],
    format='%d-%m-%Y',
    errors='coerce')
df['crawl_date'].nunique(), df['crawl_date'].unique()

(34,
 <DatetimeArray>
 ['2025-04-14 00:00:00', '2025-04-15 00:00:00', '2025-04-16 00:00:00',
  '2025-04-17 00:00:00', '2025-04-18 00:00:00', '2025-04-11 00:00:00',
  '2025-04-12 00:00:00', '2025-04-13 00:00:00', '2025-04-19 00:00:00',
  '2025-04-24 00:00:00', '2025-04-29 00:00:00', '2025-05-01 00:00:00',
  '2025-04-20 00:00:00', '2025-04-21 00:00:00', '2025-04-22 00:00:00',
  '2025-04-23 00:00:00', '2025-04-25 00:00:00', '2025-04-26 00:00:00',
  '2025-04-27 00:00:00', '2025-04-28 00:00:00', '2025-04-30 00:00:00',
  '2025-05-02 00:00:00', '2025-05-03 00:00:00', '2025-05-04 00:00:00',
  '2025-05-05 00:00:00', '2025-05-06 00:00:00', '2025-05-07 00:00:00',
  '2025-05-08 00:00:00', '2025-05-09 00:00:00', '2025-04-07 00:00:00',
  '2025-04-08 00:00:00', '2025-04-09 00:00:00', '2025-04-10 00:00:00',
  '2025-05-10 00:00:00']
 Length: 34, dtype: datetime64[ns])

In [23]:
df['days_left'] = (
    pd.to_datetime(
        df['start_day']) -
    pd.to_datetime(
        df['crawl_date'])).dt.days
df[['start_day', 'crawl_date', 'days_left']].value_counts().head()

start_day   crawl_date  days_left
2025-05-05  2025-04-22  13           132
2025-04-28  2025-04-23  5            132
            2025-04-17  11           132
            2025-04-22  6            132
2025-05-05  2025-04-20  15           131
Name: count, dtype: int64

In [24]:
df['days_left'].value_counts()

days_left
11    2431
10    2427
8     2412
9     2401
12    2392
14    2381
13    2339
5     2323
7     2313
6     2299
4     2298
15    2247
3     2207
2     2172
16    2059
1     1970
17    1940
18    1868
19    1634
20    1549
22     103
24     103
23     102
25     100
21      98
28      97
30      94
29      94
26      92
27      92
31      41
32      12
34      12
33      12
Name: count, dtype: int64

In [25]:
df.drop(columns=['start_day', 'end_day', 'crawl_date'], inplace=True)

In [26]:
df.isna().sum()

id                    0
brand                 0
price                 0
destination           0
checked_baggage    8643
hand_luggage       8643
start_hour            0
end_hour              0
trip_mins             0
is_holiday            0
days_left             0
dtype: int64

## luggage:

In [27]:
df[['hand_luggage', 'checked_baggage']].value_counts()

hand_luggage                checked_baggage  
Hành lý xách tay 7 kg       Hành lý 0 kg         19506
Hành lý xách tay 1 x 12 kg  Hành lý 23 kg         8476
                            Hành lý 1 x 23 kg     6596
Hành lý xách tay 7 kg       Hành lý 20 kg          442
Hành lý xách tay 10 kg      Hành lý 1 x 23 kg      437
                            Hành lý 23 kg          421
Hành lý xách tay 1 x 12 kg  Hành lý 0 kg           189
Hành lý xách tay 10 kg      Hành lý 0 kg             4
Name: count, dtype: int64

In [28]:
def format_lugggage(string):
    if pd.isna(string):
        return None
    number = re.findall(r'\d+', str(string))
    if len(number) >= 2:  # ve co hanh ly xach tay format theo kien hang 1x12kg
        return int(number[0]) * int(number[1])
    elif len(number) == 1:
        return int(number[0])
    else:
        return None


column = ['checked_baggage', 'hand_luggage']
for c in column:
    df[c] = df[c].apply(format_lugggage)

In [29]:
df['hand_luggage'] = df.groupby('id')['hand_luggage'] .transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan))

df['checked_baggage'] = df.groupby('id')['checked_baggage'] .transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan))

In [30]:
df[['hand_luggage', 'checked_baggage']].isnull().sum()

hand_luggage       155
checked_baggage    155
dtype: int64

In [31]:
df[df['hand_luggage'].isnull() | df['checked_baggage'].isnull()]

,id,brand,price,destination,checked_baggage,hand_luggage,start_hour,end_hour,trip_mins,is_holiday,days_left
5,CXR0002,VietJet Air,818157,Nha Trang (CXR)\nSân bay Cam Ranh,NaN,NaN,Afternoon,Afternoon,60,0,20
225,CXR0015,VietJet Air,1286535,Nha Trang (CXR)\nSân bay Cam Ranh,NaN,NaN,Afternoon,Afternoon,60,0,16
226,CXR0015,VietJet Air,1142489,Nha Trang (CXR)\nSân bay Cam Ranh,NaN,NaN,Afternoon,Afternoon,60,0,15
424,CXR0035,VietJet Air,831915,Nha Trang (CXR)\nSân bay Cam Ranh,NaN,NaN,Evening,Evening,60,1,16
508,CXR0042,VietJet Air,831915,Nha Trang (CXR)\nSân bay Cam Ranh,NaN,NaN,Evening,Evening,60,1,16
...,...,...,...,...,...,...,...,...,...,...,...
40084,PQC0121,VietJet Air,837000,Phú Quốc (PQC)\nSân bay Phú Quốc,NaN,NaN,Afternoon,Evening,60,0,16
40247,PQC0171,Vietnam Airlines,879056,Phú Quốc (PQC)\nSân bay Phú Quốc,NaN,NaN,Morning,Morning,55,0,12
40262,PQC0172,Vietnam Airlines,879056,Phú Quốc (PQC)\nSân bay Phú Quốc,NaN,NaN,Morning,Morning,55,0,13
40492,PQC0171,Vietnam Airlines,866133,Phú Quốc (PQC)\nSân bay Phú Quốc,NaN,NaN,Morning,Morning,55,0,11


In [32]:
brand_null_luggage = df[df['hand_luggage'].isnull() | df['checked_baggage'].isnull()]['brand'].unique()
brand_null_luggage

array(['VietJet Air', 'Vietnam Airlines'], dtype=object)

In [33]:
# VietJet Air
mask = (df['brand'] == 'VietJet Air') & (df['hand_luggage'].isna())
df.loc[mask, 'hand_luggage'] = 7
df.loc[mask, 'checked_baggage'] = 0

# Vietnam Airlines
mask = (df['brand'] == 'Vietnam Airlines') & (df['hand_luggage'].isna())
df.loc[mask, 'hand_luggage'] = 12
df.loc[mask, 'checked_baggage'] = 23

In [34]:
df['hand_luggage'] = df['hand_luggage'].astype(int)
df['checked_baggage'] = df['checked_baggage'].astype(int)

In [35]:
df.head()

,id,brand,price,destination,checked_baggage,hand_luggage,start_hour,end_hour,trip_mins,is_holiday,days_left
0,CXR0001,VietJet Air,1892258,Nha Trang (CXR)\nSân bay Cam Ranh,0,7,Afternoon,Afternoon,60,3,17
1,CXR0001,VietJet Air,1431413,Nha Trang (CXR)\nSân bay Cam Ranh,0,7,Afternoon,Afternoon,60,3,16
2,CXR0001,VietJet Air,1327929,Nha Trang (CXR)\nSân bay Cam Ranh,0,7,Afternoon,Afternoon,60,3,15
3,CXR0001,VietJet Air,1283783,Nha Trang (CXR)\nSân bay Cam Ranh,0,7,Afternoon,Afternoon,60,3,14
4,CXR0001,VietJet Air,1285778,Nha Trang (CXR)\nSân bay Cam Ranh,0,7,Afternoon,Afternoon,60,3,13


In [36]:
df.isna().sum()

id                 0
brand              0
price              0
destination        0
checked_baggage    0
hand_luggage       0
start_hour         0
end_hour           0
trip_mins          0
is_holiday         0
days_left          0
dtype: int64

In [38]:
df.to_csv('cleaned_file.csv', index=False)